# 01-Rag Intro

Note: Please bottom section for solutions to homework

In [24]:
import minsearch, json
import os

In [28]:
api_key=os.environ.get('OPENAI_API_KEY')

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
# keywords_field used to filter e.g., to a particular course, e.g, data-engineering zoomcamp, etc
# text_fields - used for performance search, used to search for information in the querty terms
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
q = 'the course has already started, can I still enroll?'

In [13]:
index.fit(documents)

In [19]:
# boost is used to enter importance for certain text fields
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'}, # used to restrict the answers to a particular course by filtering
    boost_dict=boost,
    num_results=5
)

# the resulting documents can be used as context when generating the prompt for input into an LLM

In [20]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [21]:
from openai import OpenAI

In [29]:
client = OpenAI(api_key=api_key)

In [32]:
# no context used in this case
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [33]:
response.choices[0].message.content

"Typically, whether you can still enroll in a course after it has started depends on the policies of the institution and the specific course in question. Here are some steps you can take to find out:\n\n1. **Check the Course Description:** Sometimes, course listings will specify if late enrollments are permitted.\n   \n2. **Contact the Instructor:** The course instructor can provide information on whether they will allow late enrollment and if any special arrangements can be made for you to catch up.\n\n3. **Reach Out to Student Services/Enrollment Office:** They can offer guidance on the institutional policies and help facilitate late enrollment if possible.\n\n4. **Review Academic Calendar Deadlines:** Make sure you understand the last date you can add a course without penalties or special permissions.\n\n5. **Consider Online or Self-Paced Courses:** If the institution doesn't allow late enrollment for standard courses, see if there are online or self-paced courses available on the s

In [42]:
# good idea to provide a prompt, e.g., "you are a course teaching assitant"
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the question.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""


In [40]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [41]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [47]:
# add the details to the prompt_template
prompt = prompt_template.format(question=q, context=context).strip()

In [48]:
print(prompt)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the question.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.


In [49]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

# Cleaning up the above code

Put in a modular form

In [52]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'}, # used to restrict the answers to a particular course by filtering
        boost_dict=boost,
        num_results=5
    )

    return results

In [75]:
def build_prompt(query, search_results):
    prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the question.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    # add the details to the prompt_template
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [61]:
build_prompt(query, search_results)

"You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the question.\nIf the CONTEXT doesn't contain the answer, output NONE\n\nQUESTION: the course has already started, can I still enroll?\n\nCONTEXT:\nsection: Workshop 1 - dlthub\nquestion: How do I install the necessary dependencies to run the code?\nanswer: Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).\n\nsection: General course-related questions\nquestion: How do I use Git / GitHub for this course?\nanswer: After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repos

In [62]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)
    return response.choices[0].message.content

In [78]:
#query = "the course already started. Can I still enroll" 

query = "How do I run kafka?"
# the below functionality allows us to replace search and llm functionalities
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [79]:
rag(query)

'To run Kafka, follow the instructions provided for different scenarios:\n\n1. **For Java Kafka Producer/Consumer/KStreams:**\n   - Navigate to the project directory.\n   - Run the following command to execute the Java Kafka producer (replace `<jar_name>` with your specific jar file name):\n     ```sh\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n2. **For Python Kafka Producer:**\n   - If you encounter a "Module “kafka” not found" error, create a virtual environment and install the required packages:\n     ```sh\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n   - To activate the virtual environment (run it every time you need the virtual env):\n     ```sh\n     source env/bin/activate\n     ```\n   - To deactivate the virtual environment:\n     ```sh\n     deactivate\n     ```\n   - Note: On Windows, the activation command will be slightly different:\n     ``

In [80]:
rag("the course has already started, can I still enroll?")
    

'Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects.'

In [81]:
from elasticsearch import Elasticsearch

In [82]:
es_client = Elasticsearch('http://localhost:9200')

In [86]:
print(es_client.info())

{'name': 'c4b35a7585a2', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'eVo-6_41Q5q4qqEGBNxcIA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [87]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [88]:
from tqdm.auto import tqdm

/home/onwunalu/.pyenv/versions/llm-zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [92]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:19<00:00, 48.10it/s]


In [93]:
query = "I just discovered the course. Can I still join it?"

In [124]:
# question^3 means question text is 3 times more important than text and section texts
def elastic_search(query):
    search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "data-engineering-zoomcamp"
                        }
                    }
                }
            }
        }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [115]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the 

In [116]:
def rag_with_es(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [117]:
rag_with_es(query)

"Yes, you can still join the course. Even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

# Homework Solutions

## Homework 1 Question 1

In [ ]:
# info for elasticsearch docker run:
{
  "name" : "c4b35a7585a2",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "eVo-6_41Q5q4qqEGBNxcIA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Homework Question 3

create another index baed on text and question fields


In [228]:

index_settingsv2 = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_namev2 = "course-questionsv2"
es_client.indices.create(index=index_namev2, body=index_settingsv2)

for doc in tqdm(documents):
    es_client.index(index=index_namev2, document=doc)


'\nindex_settingsv2 = {\n    "settings": {\n        "number_of_shards": 1,\n        "number_of_replicas": 0\n    },\n    "mappings": {\n        "properties": {\n            "text": {"type": "text"},\n            "question": {"type": "text"},\n            "course": {"type": "keyword"} \n        }\n    }\n}\n\nindex_namev2 = "course-questionsv2"\nes_client.indices.create(index=index_namev2, body=index_settingsv2)\n\nfor doc in tqdm(documents):\n    es_client.index(index=index_namev2, document=doc)\n'

In [127]:
# use only 'question' and 'text' fields
"""
# from stackoverflow search: 
# best_fields:
best_fields is most useful when you are searching for multiple words best found in the same field.

# for most_fields
A little remark for most_fields - it uses score from both fields. and divide it on fields count

In your example: ( Score from title1 + score from title2 ) / 2

"""
index_settingsx = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "type": "best_fields",
        }
    }
    
}

index_name = "course-questions"
#es_client.indices.create(index=index_name, body=index_settings)

In [181]:
def elastic_searchv2(query):
    search_query = {
            "size": 10,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text",],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "data-engineering-zoomcamp"
                        }
                    }
                }
            }
        }

    response = es_client.search(index=index_namev2, body=search_query)
    result_docs = []
    result_scores = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        result_scores.append(hit['_score'])
    
    return result_docs, result_scores

In [142]:
def rag_with_esv2(query):
    search_results = elastic_searchv2(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [182]:
query =  "How do I execute a command in a running docker container?"
#rag_with_esv2(query)
search_results, result_scores=elastic_searchv2(query)
result_scores[0]

75.54128

In [153]:
result_scores

[75.66571,
 75.66571,
 44.00519,
 44.00519,
 38.7155,
 38.7155,
 38.36773,
 38.36773,
 35.97185,
 35.97185]

## Homework 1, Question 4:  Filtering


In [183]:
def elastic_searchv3(query, course="data-engineering-zoomcamp"):
    search_query = {
            "size": 3,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text",],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": course
                        }
                    }
                }
            }
        }

    response = es_client.search(index=index_namev2, body=search_query)
    result_docs = []
    result_scores = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        result_scores.append(hit['_score'])
    
    return result_docs, result_scores

In [184]:
query

'How do I execute a command in a running docker container?'

In [185]:
es_docs, es_doc_scores = elastic_searchv3(query, course='machine-learning-zoomcamp')

In [186]:
es_docs, es_doc_scores


([{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I copy files from my local machine to docker contain

In [187]:
es_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Homework 1 Q5. Building a prompt

In [212]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""
for doc in es_docs:
    #context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    context = context + context_template.format(question=doc['question'], text=doc['text'])
    

In [213]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [214]:
 # add the details to the prompt_template
prompt = prompt_template.format(question=query, context=context)

In [211]:
len(prompt)

1458

# Homework 1 Q6: Tokens

In [219]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")


In [222]:
def num_tokens_from_string(string: str, encoding_name: str="gpt-4o") -> int:
    """Returns the number of tokens in a text string."""
    #encoding = tiktoken.get_encoding(encoding_name)
    encoding = tiktoken.encoding_for_model("gpt-4o")
    
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [223]:
num_tokens_from_string(prompt)

321

In [225]:
len(encoding.encode(prompt))

321